<a href="https://colab.research.google.com/github/luifo/IA-image-super-resolution-tensorflow/blob/test/flower_id_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Ruta en googledrive
PATH = "/content/drive/MyDrive/102flowers"

# Ruta de datos de entrada
IN_PATH = PATH + '/jpg001'
# Ruta de datos de salida
OUT_PATH = PATH + '/jpg002'
# Ruta de check points
CK_PATH = PATH + '/checkpints'

# usando codigo de consola listando el directorio de entrada
img_urls = !ls -1 "{IN_PATH}"

# Entrenando con 500 imagenes
n = 500
train_n = round(n*0.80)
# listado randomizado
rand_urls = np.copy(img_urls)

np.random.seed(23) #solo para el tutorial
np.random.shuffle(rand_urls)

# Particion train test}
tr_urls = rand_urls[:train_n]
ts_urls = rand_urls[train_n:n]

print(len(img_urls), len(tr_urls), len(ts_urls))

8199 400 100


In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

def resize(inimg, targimg, height, width)
  inimg = tf.image.resize(inimg, [height, width])
  targimg = tf.image.resize(targimg, [height, width])

  return inimg, targimg



# New Section